# Dataset fingerprints

## Imports

In [3]:
from pathlib import Path
import pickle

import pandas as pd

## IO paths

In [4]:
path_to_results = Path('.') / '..' / 'results' 

## Load metadata for failed fingerprint generation

In [9]:
with open(path_to_results / 'fingerprints_error_entries.p', 'rb') as f:
    errors = pickle.load(f)
    
errors = pd.DataFrame(errors)

In [14]:
len(errors)

3

In [10]:
errors

,Unnamed: 0,index,kinase,family,groups,pdb_id,chain,alternate_model,species,ligand_orthosteric_name,...,dfg,ac_helix,rmsd1,rmsd2,qualityscore,pocket,resolution,missing_residues,missing_atoms,full_ifp
1000,1005,1259,CHK1,CAMKL,CAMK,4hyh,A,B,Human,"2-(6-methoxy-1-oxo-1,3-dihydro-2H-isoindol-2-y...",...,in,in,0.779,2.097,8.0,QTLGEGAYGEVQLVAVKIVNIKKEICINKMLNENVVKFYGHYLFLE...,1.7,0,0,0000000000000010000000000000000000000000000000...
1002,1007,1317,CHK1,CAMKL,CAMK,4jik,A,-,Human,2-(4-chlorophenyl)-8-[(3S)-piperidin-3-ylamino...,...,in,in,0.779,2.028,8.7,QTLGEGA_GEVQLVAVKIVNIKKEICINKMLNENVVKFYGHYLFLE...,1.9,1,9,0000000000000010000000000000100000010000000000...
2479,2490,747,LOK,STE20,STE,2j7t,A,A,Human,"(3Z)-N-(3-CHLOROPHENYL)-3-({3,5-DIMETHYL-4-[(4...",...,in,in,0.776,2.105,8.0,GELGDGAFGKVYKAAAKVIDYIVEIEILATCDPYIVKLLGAWIMIE...,2.0,0,26,0000000000000010000000000000000000000000000000...


## Load fingerprints

In [12]:
fingerprints = pd.read_csv(path_to_results / 'fingerprints.csv')

In [13]:
fingerprints.shape

(332690, 15)

In [15]:
(3917- len(errors)) * 85

332690

In [16]:
fingerprints.head()

,Unnamed: 0,size,hbd,hba,charge,aromatic,aliphatic,sco,exposure,distance_to_centroid,distance_to_hinge_region,distance_to_dfg_region,distance_to_front_pocket,metadata_index,molecule_code
0,1,2.0,0.0,2.0,-1.0,0.0,0.0,94.16,0.296296,17.19,12.80,19.08,14.13,0,HUMAN/AAK1_4wsq_altA_chainB
1,2,1.0,0.0,0.0,0.0,0.0,1.0,21.29,0.344828,15.09,11.64,16.48,11.69,0,HUMAN/AAK1_4wsq_altA_chainB
2,3,2.0,0.0,0.0,0.0,0.0,1.0,27.98,0.750000,12.41,9.24,15.13,8.24,0,HUMAN/AAK1_4wsq_altA_chainB
3,4,1.0,0.0,0.0,0.0,0.0,1.0,22.51,0.451613,11.80,10.34,13.67,7.22,0,HUMAN/AAK1_4wsq_altA_chainB
4,5,2.0,0.0,2.0,-1.0,0.0,0.0,25.87,0.142857,12.80,12.41,12.79,9.24,0,HUMAN/AAK1_4wsq_altA_chainB
